# MLflow Parent-Child Runs Demo

This notebook demonstrates how to:
1. Create a parent run in a "train" pipeline and save run info to JSON
2. Use the saved run info in an "evaluation" pipeline to create nested child runs

This pattern is useful when you have separate pipelines that need to be linked together.

**Key Features:**
- ✅ **Proper parent-child relationship** using `nested=True`
- ✅ **JSON-based run information persistence** between pipelines
- ✅ **Type hints** for all functions
- ✅ **Comprehensive logging** of parameters, metrics, and artifacts

**Real-world Applicability:**
This pattern is perfect for scenarios where:
- Training and evaluation happen in separate pipeline stages
- You need to track related runs across different execution contexts
- Multiple evaluation types need to be linked to a single training run
- You want to maintain run relationships across different systems/environments

The notebook is ready to run and will create a complete MLflow experiment showing the parent-child relationship with proper metric tracking and artifact logging! 🚀


In [5]:
import json
import joblib
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime

from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from typing import Dict, Any


## Setup MLflow


In [6]:
# Set up MLflow
MLFLOW_TRACKING_URI = "http://localhost:5001"
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

client = MlflowClient()
print(f"MLflow tracking URI: {client.tracking_uri}")

# Set experiment
EXPERIMENT_NAME = "4-train-eval-with-nested-runs"
mlflow.set_experiment(EXPERIMENT_NAME)

print(f"Experiment: {EXPERIMENT_NAME}")

MLflow tracking URI: http://localhost:5001
Experiment: 4-train-eval-with-nested-runs


## Load and Prepare Data



In [7]:
# Load the bike sharing dataset
raw_data = pd.read_csv("../data/raw_data.csv")

# Define features and target
target = 'cnt'
numerical_features = ['temp', 'atemp', 'hum', 'windspeed', 'mnth', 'hr', 'weekday']
categorical_features = ['season', 'holiday', 'workingday']
features = numerical_features + categorical_features

# Prepare the data
X = raw_data[features]
y = raw_data[target]

print(f"Dataset shape: {X.shape}")
print(f"Features: {features}")
print(f"Target: {target}")


Dataset shape: (17379, 10)
Features: ['temp', 'atemp', 'hum', 'windspeed', 'mnth', 'hr', 'weekday', 'season', 'holiday', 'workingday']
Target: cnt


# 1. Train Pipeline - Create Parent Run

This section simulates the training pipeline where we:
- Create a parent MLflow run
- Train a model and log training metrics
- Save the model
- Save run information to a JSON file for later use



In [8]:
def train_pipeline() -> tuple[Dict[str, Any], tuple]:
    """
    Train pipeline that creates a parent MLflow run and saves run info.
    
    Returns:
        Tuple of (run_info_dict, test_data_tuple)
    """
    # Split data for training
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    print(f"Training set size: {X_train.shape}")
    print(f"Test set size: {X_test.shape}")
    
    # Start parent MLflow run
    with mlflow.start_run(run_name="Training-Pipeline") as parent_run:
        
        print(f"\n=== TRAINING PIPELINE ===")
        print(f"Parent Run ID: {parent_run.info.run_id}")
        print(f"Parent Run Name: {parent_run.info.run_name}")
        print(f"Experiment ID: {parent_run.info.experiment_id}")
        
        # Log training parameters
        model_params = {
            "n_estimators": 100,
            "random_state": 42,
            "max_depth": 10
        }
        
        mlflow.log_params(model_params)
        mlflow.log_param("train_size", len(X_train))
        mlflow.log_param("test_size", len(X_test))
        mlflow.log_param("features", ", ".join(features))
        
        # Train model
        print("\nTraining model...")
        model = ensemble.RandomForestRegressor(**model_params)
        model.fit(X_train, y_train)
        
        # Make predictions and calculate training metrics
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)
        
        # Training metrics
        train_mse = mean_squared_error(y_train, y_train_pred)
        train_mae = mean_absolute_error(y_train, y_train_pred)
        train_r2 = r2_score(y_train, y_train_pred)
        
        # Log training metrics
        training_metrics = {
            "mse": train_mse,
            "mae": train_mae,
            "r2": train_r2,
        }
        
        mlflow.log_metrics(training_metrics)
        
        # Save and log model
        model_path = Path("../models/trained_model.joblib")
        model_path.parent.mkdir(exist_ok=True)
        joblib.dump(model, model_path)
        
        # Log model to MLflow
        mlflow.sklearn.log_model(
            model, 
            "model",
            registered_model_name="BikeSharing-RandomForest"
        )
        
        # Log model artifact
        mlflow.log_artifact(str(model_path), "model_files")
        
        # Prepare run information to save
        run_info = {
            "parent_run_id": parent_run.info.run_id,
            "parent_run_name": parent_run.info.run_name,
            "experiment_id": parent_run.info.experiment_id,
            "experiment_name": EXPERIMENT_NAME,
            "model_path": str(model_path),
            "training_completed_at": datetime.now().isoformat(),
            "training_metrics": training_metrics,
            "model_params": model_params,
            "mlflow_tracking_uri": MLFLOW_TRACKING_URI
        }
        
        print(f"\n🏃 View run at: {MLFLOW_TRACKING_URI}/#/experiments/{parent_run.info.experiment_id}/runs/{parent_run.info.run_id}")
        
        return run_info, (X_test, y_test)  # Return test data for evaluation

# Execute training pipeline
run_info, test_data = train_pipeline()


2025/06/11 23:04:24 INFO mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics. Set logger level to DEBUG for more details.
2025/06/11 23:04:24 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


Training set size: (13903, 10)
Test set size: (3476, 10)

=== TRAINING PIPELINE ===
Parent Run ID: 4ca6339163b94b3abc69b6c2a6fd9a36
Parent Run Name: Training-Pipeline
Experiment ID: 424352721332353736

Training model...


2025/06/11 23:04:27 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/06/11 23:04:27 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'BikeSharing-RandomForest' already exists. Creating a new version of this model...
2025/06/11 23:04:27 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: BikeSharing-RandomForest, version 4
Created version '4' of model 'BikeSharing-RandomForest'.
2025/06/11 23:04:27 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2025/06/11 23:04:27 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!



🏃 View run at: http://localhost:5001/#/experiments/424352721332353736/runs/4ca6339163b94b3abc69b6c2a6fd9a36
🏃 View run Training-Pipeline at: http://localhost:5001/#/experiments/424352721332353736/runs/4ca6339163b94b3abc69b6c2a6fd9a36
🧪 View experiment at: http://localhost:5001/#/experiments/424352721332353736


## Save Run Information to JSON File



In [9]:
# Save run information to JSON file
run_info_file = Path("mlflow_run_info.json")

with open(run_info_file, 'w') as f:
    json.dump(run_info, f, indent=2)

print(f"✅ Run information saved to: {run_info_file}")
print(f"\nSaved run info:")
print(json.dumps(run_info, indent=2))


✅ Run information saved to: mlflow_run_info.json

Saved run info:
{
  "parent_run_id": "4ca6339163b94b3abc69b6c2a6fd9a36",
  "parent_run_name": "Training-Pipeline",
  "experiment_id": "424352721332353736",
  "experiment_name": "4-train-eval-with-nested-runs",
  "model_path": "../models/trained_model.joblib",
  "training_completed_at": "2025-06-11T23:04:27.932866",
  "training_metrics": {
    "mse": 4188.159720650497,
    "mae": 42.91092139773795,
    "r2": 0.873857924209189
  },
  "model_params": {
    "n_estimators": 100,
    "random_state": 42,
    "max_depth": 10
  },
  "mlflow_tracking_uri": "http://localhost:5001"
}


# 2. Evaluation Pipeline - Create Child Runs

This section simulates a separate evaluation pipeline that:
- Loads the run information from the JSON file
- Creates nested child runs under the parent
- Performs different types of evaluation
- Logs evaluation metrics to the child runs



In [10]:
def load_run_info(file_path: str) -> Dict[str, Any]:
    """
    Load run information from JSON file.
    
    Args:
        file_path: Path to the JSON file containing run information
        
    Returns:
        Dictionary containing run information
    """
    with open(file_path, 'r') as f:
        return json.load(f)

# Load the saved run information
print("=== EVALUATION PIPELINE ===")
print("Loading run information from JSON file...")

loaded_run_info = load_run_info("mlflow_run_info.json")

print(f"✅ Loaded run info for parent run: {loaded_run_info['parent_run_id']}")
print(f"Parent run name: {loaded_run_info['parent_run_name']}")
print(f"Training completed at: {loaded_run_info['training_completed_at']}")


=== EVALUATION PIPELINE ===
Loading run information from JSON file...
✅ Loaded run info for parent run: 4ca6339163b94b3abc69b6c2a6fd9a36
Parent run name: Training-Pipeline
Training completed at: 2025-06-11T23:04:27.932866


In [11]:
def evaluation_pipeline(run_info: Dict[str, Any], test_data: tuple) -> None:
    """
    Evaluation pipeline that creates child runs under the parent.
    
    Args:
        run_info: Dictionary containing parent run information
        test_data: Tuple of (X_test, y_test) for evaluation
    """
    # Set the tracking URI from the saved info
    mlflow.set_tracking_uri(run_info['mlflow_tracking_uri'])
    
    # Load the trained model
    model = joblib.load(run_info['model_path'])
    X_test, y_test = test_data
    
    print(f"\nLoaded model from: {run_info['model_path']}")
    print(f"Evaluation dataset size: {X_test.shape}")
    
    # Use the parent run ID to create nested runs
    parent_run_id = run_info['parent_run_id']
    
    # Evaluation 1: Standard Metrics Evaluation
    with mlflow.start_run(
        run_id=parent_run_id,  # Resume the parent run
        nested=False  # We're resuming, not nesting yet
    ):
        with mlflow.start_run(
            run_name="Evaluation-Pipeline", 
            nested=True
        ) as eval_run_1:
            
            print(f"\n--- Standard Evaluation (Child Run 1) ---")
            print(f"Child Run ID: {eval_run_1.info.run_id}")
            print(f"Parent Run ID: {parent_run_id}")
            
            # Make predictions
            y_pred = model.predict(X_test)
            
            # Calculate and log evaluation metrics
            eval_metrics = {
                "mse": mean_squared_error(y_test, y_pred),
                "mae": mean_absolute_error(y_test, y_pred),
                "r2": r2_score(y_test, y_pred)
            }
            mlflow.log_metrics(eval_metrics)
            
            # Log evaluation parameters
            mlflow.log_param("evaluation_type", "standard_metrics")
            mlflow.log_param("evaluation_dataset_size", len(X_test))
            mlflow.log_param("evaluation_timestamp", datetime.now().isoformat())
            
# Execute evaluation pipeline
evaluation_pipeline(loaded_run_info, test_data)


2025/06/11 23:04:27 INFO mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics. Set logger level to DEBUG for more details.
2025/06/11 23:04:27 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
2025/06/11 23:04:27 INFO mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics. Set logger level to DEBUG for more details.
2025/06/11 23:04:27 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
2025/06/11 23:04:28 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2025/06/11 23:04:28 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!
2025/06/11 23:04:28 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2025/06/11 23:04:28 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!



Loaded model from: ../models/trained_model.joblib
Evaluation dataset size: (3476, 10)

--- Standard Evaluation (Child Run 1) ---
Child Run ID: 119598537c634b97bd090938b45c5db6
Parent Run ID: 4ca6339163b94b3abc69b6c2a6fd9a36
🏃 View run Evaluation-Pipeline at: http://localhost:5001/#/experiments/424352721332353736/runs/119598537c634b97bd090938b45c5db6
🧪 View experiment at: http://localhost:5001/#/experiments/424352721332353736
🏃 View run Training-Pipeline at: http://localhost:5001/#/experiments/424352721332353736/runs/4ca6339163b94b3abc69b6c2a6fd9a36
🧪 View experiment at: http://localhost:5001/#/experiments/424352721332353736
